# MFDP. RecSys

In [71]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Необходимо реализовать baseline для задачи рекомендации контента. Пока что не будем учитывать ни время, которое есть у клиента (потому что пока таких данных нет), ни прочие данные о клиентах. Простой baseline, как отправная точка, для разработки сервиса.

In [79]:
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm

from src.models.BaseModel import BaseRecommendModel
from src.metrics import average_precision, average_single_precision

In [3]:
np.random.seed(42)

In [4]:
data_path = Path('../data')

In [5]:
df_train = pd.read_parquet(data_path / 'df_train.parquet')

In [6]:
users_bd = pd.read_parquet(data_path / 'users_bd.parquet')

In [7]:
model = BaseRecommendModel()

In [8]:
model.fit(df_train, history=users_bd)

In [9]:
df_test = pd.read_parquet(data_path / 'df_test.parquet')

In [10]:
# Рекомендуем всем одинаково, потому что генерить рекомендации, проверяя историю, для 1 000 000 строчек дорого по времени
df_test['recomendation'] = ', '.join(model.recommend('', 20, use_history=False))

In [11]:
%%time
df_test.apply(lambda x: average_single_precision(x['item'], x['recomendation'].split(', ')), axis=1).mean()

CPU times: total: 34.5 s
Wall time: 34.9 s


0.002971727412521274

Baseline score составил 0.00297

In [12]:
1/(0.00297 * 20)

16.835016835016834

В среднем то, что понравится пользователю (из этих датасетов) находится на 16 позиции, что в принципе очень неплохо)

# Сохранение модели для использования

In [80]:
import pickle

In [81]:
model = BaseRecommendModel()

In [82]:
model.fit(df_train)

In [84]:
model.recommend('', 20, user_history=['k8636', 'k1844', 'k4457'])

array(['k15297', 'k13865', 'b0971880107', 'k10440', 'k3734', 'k9728',
       'k142', 'k11237', 'k7417', 'k6809', 'k14431', 'k7571', 'k3182',
       'k1819', 'k7626', 'k16166', 'k4495', 'k9169', 'k5693', 'k14461'],
      dtype=object)

In [85]:
model_path = Path('../src/models')

In [86]:
with open(model_path / 'base_model.pickle', 'wb') as f:
    pickle.dump(model, f)